In [1]:
import re
import os
import time
import pickle
import IPython
import sklearn
import numpy as np
import kmapper as km
import pandas as pd
import seaborn as sns
from umap import UMAP

from tqdm import tqdm
from collections import defaultdict
from sklearn.manifold import t_sne, isomap

from sklearn import datasets
from sklearn.cluster import DBSCAN

from sklearn.decomposition import TruncatedSVD
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


import cx_Oracle

# oracle
oracle_user = "ro_user"
oracle_pass = "ro_user"
oracle_scheme = "aml_evraz"
oracle_host = "192.168.101.13/rnd"
oracle_connection = cx_Oracle.connect(oracle_user, oracle_pass, oracle_host, encoding = "UTF-8", nencoding = "UTF-8")
oracle_cursor = oracle_connection.cursor()


%matplotlib inline
sns.set(style='white', rc={'figure.figsize':(12,8)})

In [2]:
class Timer:
    def __init__(self, msg='operation', verbose=True):
        self.msg = msg
        self.verbose = verbose       
    def __enter__(self, ):
        self.start = time.clock()
        return self

    def __exit__(self, *args):
        self.end = time.clock()
        self.interval = self.end - self.start
        if self.verbose:
            print('{} took {:.3f}s'.format(self.msg, self.interval), flush=True)


def df_categorical_variables_stat(df, max_cardinality=200):
    for colname in df.columns:
        print('next_column: {}'.format(colname))
        uniq_vals = df[colname].unique()
        nunique = len(uniq_vals)
        print("# of uniqs: {}".format(nunique))
        if nunique < max_cardinality:
            print('Viable Categorical. Value counts:')
            print(df[colname].value_counts(dropna=False))
        else:
            print('High cardinality/Non categorical')
        print((('-' * 80) + '\n')*3)

In [3]:
try:
    del susp_ops
except:
    pass

try:
    del susp_members
except:
    pass


try:
    del off_ops
except:
    pass

try:
    del off_members
except:
    pass

with Timer('read susp_ops', True):
    susp_ops = pd.read_csv('../../data/susp_ops.csv')
with Timer('read susp_members', True):
    susp_members = pd.read_csv('../../data/susp_members.csv')
with Timer('read off_ops', True):
    off_ops = pd.read_csv('../../data/off_ops.csv')
with Timer('read off_ops', True):
    off_members = pd.read_csv('../../data/off_members.csv')

read susp_ops took 3.212s


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4,9,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


read susp_members took 3.570s


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,3,21,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,3,8,9,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


read off_ops took 41.536s


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,3,5,11,17,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


read off_ops took 36.941s


In [4]:
with Timer('process off_ops', True):
    off_ops.loc[:, 'P_OPERATIONDATETIME'] = pd.to_datetime(off_ops['P_OPERATIONDATETIME'])
    off_ops.sort_values(by='P_OPERATIONDATETIME', kind='mergesort', inplace=True)

process off_ops took 4.840s


### Helper functions

* process_client_indices() --- parse numeric indices with small typo errors to ints, and encode non-digit ids with negative numbers
* flatten_by_column() --- each operation_id in OFF_OPERATIONS has 1-3 linked members with disjoint P_CLIENTROLE values. We want unique operation_id per row, with linked members with diferent roles spreading into a row columns.
* join_ops_with_flatten_members() --- join operations dataframe with flattened operation id dataframe

In [5]:
def process_client_indices(client_indices, trivial_to_nontrivial=None):
    if trivial_to_nontrivial is None:
        trivial_ids_to_nontrivial = defaultdict(lambda: -len(trivial_ids_to_nontrivial) - 2)

    nontrivial_clients_ids = client_indices.copy()
    nontrivial_clients_ids[nontrivial_clients_ids.isnull()] = -1

    nontrivial_clients_ids = nontrivial_clients_ids.map(str)

    nontrivial_ids = nontrivial_clients_ids.map(lambda s: s.strip())

    is_trivial = nontrivial_ids.map(lambda s: len(re.findall('^([\d]+|-[\d]+)', s)) == 0)
    trivial_ids = nontrivial_ids[is_trivial]

    for ti in trivial_ids:
        trivial_ids_to_nontrivial[ti]
    trivial_ids_to_nontrivial
    nontrivial_ids[is_trivial] = trivial_ids.map(lambda s: str(trivial_ids_to_nontrivial[s]))

    nontrivial_ids = nontrivial_ids.map(lambda s: re.findall('^([\d]+|-[\d]+)', s)[0]).map(int)
    return nontrivial_ids, trivial_to_nontrivial


def flatten_by_column(df, to_flatten, flatten_by, group_by, names_flatten_by=None):
    if names_flatten_by is None:
        names_flatten_by = {
            uniq_val: '{}={}'.format(flatten_by, uniq_val)
            for uniq_val in df[flatten_by].unique()
        }
    #column_to_flatten_by = 'P_CLIENTROLE'
    #column_to_group_by = 'P_SUSPICIOUSOPERATIONID'
    #column_to_flatten = 'P_CLIENTID'
    column_to_flatten_by = flatten_by
    column_to_flatten = to_flatten
    column_to_group_by = group_by
    frames_to_join = [
        df[[column_to_group_by, column_to_flatten]][df[column_to_flatten_by] == uniq_val]
        for uniq_val in df[column_to_flatten_by].unique()
    ]

    for frame, uniq_val in zip(frames_to_join, df[flatten_by].unique()):
        frame.rename(columns={column_to_flatten: names_flatten_by[uniq_val]}, inplace=True)
        frame.set_index(column_to_group_by, inplace=True)

    res = frames_to_join[0].join(frames_to_join[1:], how='outer')
    return res


def join_ops_with_flatten_members(ops, flatten_ops_with_members, id_colname='ID'):
    ops_to_join = ops.set_index(id_colname, drop=False)
    joined_ops = ops_to_join.join(flatten_ops_with_members, how='left')
    joined_ops.reset_index(drop=True, inplace=True)
    return joined_ops

### Parse P_CLIENTID for offline members

In [6]:
with Timer('off_members client_ids processing', True):
    off_members.loc[:, 'P_CLIENTID'], trivial_ids_to_nontrivial = process_client_indices(off_members.P_CLIENTID)

off_members client_ids processing took 48.448s


### P_CLIENTROLE has only 3 values and not more than 3 P_CLIENTID with different P_CLIENTROLE corresponds to a single P_OPERATIONID value. So flatten them by P_CLIENTROLE

In [7]:
try:
    del off_res
except:
    pass

names_by_clientrole = {
    1: 'id_from',
    2: 'id_to',
    3: 'id_susp_3',
    4: 'id_susp_4',
    5: 'id_beneficial',
    6: 'id_susp_6'
}

with Timer('flatten grouped P_OPERATIONID with P_CLIENTID by P_CLIENTROLE', True):
    off_res = flatten_by_column(off_members, 'P_CLIENTID', 'P_CLIENTROLE', 'P_OPERATIONID', names_by_clientrole)

off_res = off_res.reset_index().drop_duplicates().set_index('P_OPERATIONID', drop=True)
try:
    del joined_offline_ops
except:
    pass

with Timer('join offline operations with flattened members info', True):
    joined_offline_ops = join_ops_with_flatten_members(off_ops, off_res)

flatten grouped P_OPERATIONID with P_CLIENTID by P_CLIENTROLE took 19.120s
join offline operations with flattened members info took 6.706s


### Make 0-based indices for every operation and client we met

In [8]:
try:
    del operationid_counter
except:
    pass

try:
    del operationid_inv_counter
except:
    pass

with Timer('building operation_id counter', True):
    if os.path.exists('operationid_counter.pickle'):
        with open('operationid_counter.pickle', 'rb') as handle:
            operationid_counter = defaultdict(lambda: len(operationid_counter), pickle.load(handle))
    else:
        operationid_counter = defaultdict(lambda: len(operationid_counter))
        with open('operationid_counter.pickle', 'wb') as handle:
            pickle.dump(dict(operationid_counter), handle, protocol=pickle.HIGHEST_PROTOCOL)
    joined_offline_ops.loc[:, 'ID'] = joined_offline_ops['ID'].map(operationid_counter.__getitem__)
    operationid_inv_counter = {v: k for k, v in operationid_counter.items()}

building operation_id counter took 10.409s


In [9]:
try:
    del clientid_counter
except:
    pass

try:
    del clientid_inv_counter
except:
    pass

with Timer('building client_id counter', True):
    if os.path.exists('clientid_counter.pickle'):
        with open('clientid_counter.pickle', 'rb') as handle:
            clientid_counter = defaultdict(lambda: len(clientid_counter), pickle.load(handle))
    else:
        clientid_counter = defaultdict(lambda: len(clientid_counter))
        for row_num, row in tqdm(joined_offline_ops[['ID', 'id_from', 'id_to', 'id_beneficial']].iterrows()):
            for client_id in [row.id_from, row.id_to, row.id_beneficial]:
                if np.isfinite(client_id):
                    clientid_counter[client_id]
        with open('clientid_counter.pickle', 'wb') as handle:
            pickle.dump(dict(clientid_counter), handle, protocol=pickle.HIGHEST_PROTOCOL)
    clientid_inv_counter = {v: k for k, v in clientid_counter.items()}

building client_id counter took 1.333s


### set 'target' column with susp_ops P_SENDTOKFMBOOL values. Normal transactions will have value -1

In [10]:
susp_ops = susp_ops[~susp_ops.P_OFFLINEOPERATIONID.isnull()].copy()
susp_ops.loc[susp_ops.P_OPERATION_LIST.isnull(), 'P_OPERATION_LIST'] = ''

In [11]:
#joined_offline_ops.loc[:, 'P_OPERATIONDATETIME'] = pd.to_datetime(joined_offline_ops.P_OPERATIONDATETIME)
joined_offline_ops.set_index('ID', drop=False, inplace=True)

In [12]:
joined_offline_ops['raw_id_from'] = joined_offline_ops.id_from.map(clientid_counter.get)
joined_offline_ops['raw_id_to'] = joined_offline_ops.id_to.map(clientid_counter.get)
joined_offline_ops['raw_id_beneficial'] = joined_offline_ops.id_beneficial.map(clientid_counter.get)
joined_offline_ops['seconds_from_start'] = (joined_offline_ops.P_OPERATIONDATETIME - \
                                            joined_offline_ops.P_OPERATIONDATETIME.min()).dt.total_seconds()

In [13]:
with Timer('setting proper labels for learning'):
    target_mapped = defaultdict(lambda: -1)

    for i, row in tqdm(susp_ops[['P_OFFLINEOPERATIONID', 'P_SENDTOKFMBOOL']].iterrows()):
        off_op_id, to_kfm = operationid_counter[int(row.P_OFFLINEOPERATIONID)], row.P_SENDTOKFMBOOL
        if off_op_id not in target_mapped:
            target_mapped[off_op_id] = 0 if to_kfm == 2 else 1
        else:
            cur_target = target_mapped[off_op_id]
            if cur_target == 0 and to_kfm != 2:
                target_mapped[off_op_id] = 1

401500it [00:22, 18117.38it/s]

setting proper labels for learning took 22.347s


In [14]:
joined_offline_ops['target'] = joined_offline_ops.ID.map(target_mapped.__getitem__)

In [20]:
selected_flow_columns = [
    'ID',
    'P_OPERATIONDATETIME',
    'seconds_from_start',
    'P_BASEAMOUNT',
    'raw_id_from',
    'raw_id_to',
    'target'
]

selected_scoring_columns = [
    'ID',
    'P_ISSUEDBID',
    'P_BASEAMOUNT',
    'P_BRANCH',
    'P_CURRENCYCODE',
    'P_EKNPCODE',
    'P_DOCCATEGORY',
    'P_KFM_OPER_REASON',
    'P_BS_OPER_TYPE',
    'P_WAS_SEND',
    'target'
]

In [21]:
trans_flow = joined_offline_ops[selected_flow_columns].copy()

In [26]:
len(trans_flow.seconds_from_start.unique())

1192296

In [27]:
len(trans_flow)

8222263